In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import json
from bs4 import BeautifulSoup
import numpy as np
import requests
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
with open('vetting_playlist.json', 'r') as file:
    data1 = json.load(file)

In [3]:
#data1.keys()
#This is for someone else

In [4]:
file_paths = [
    'mpd.slice.15000-15999.json',
    'mpd.slice.16000-16999.json',
    'mpd.slice.17000-17999.json',
    'mpd.slice.18000-18999.json',
    'mpd.slice.19000-19999.json',
    'mpd.slice.20000-20999.json',
    'mpd.slice.21000-21999.json',
    'mpd.slice.22000-22999.json',
    'mpd.slice.23000-23999.json',
    'mpd.slice.24000-24999.json',
    'mpd.slice.25000-25999.json',
    'mpd.slice.26000-26999.json',
    'mpd.slice.27000-27999.json',
    'mpd.slice.28000-28999.json',
    'mpd.slice.29000-29999.json',
    'mpd.slice.30000-30999.json',
]

all_data = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        for playlist in data['playlists']:
            playlist_name = playlist['name']
            for track in playlist['tracks']:
                all_data.append({
                    "playlist_name": playlist_name,
                    "position_in_playlist": track["pos"],
                    "track_id": track['track_uri'],
                    "track_name": track['track_name'],
                    "artist_name": track['artist_name'],
                    "duration_ms": track['duration_ms'],
                    "album_name": track['album_name']
                })
df = pd.DataFrame(all_data)

In [18]:
# Count how many playlists each track appears in
track_counts = df.groupby(['track_id', 'track_name', 'artist_name']).size().reset_index(name='count')

In [20]:
# Normalize counts to a popularity score (1-10)
min_count = track_counts['count'].min()
max_count = track_counts['count'].max()
track_counts['popularity_score'] = (
    ((track_counts['count'] - min_count) / (max_count - min_count) * 9 + 1
).astype(int))

In [21]:
# Merge popularity scores back into the original DataFrame
df_with_popularity = df.merge(
    track_counts[['track_id', 'popularity_score']],
    on='track_id',
    how='left'
)

In [29]:
df_with_popularity['popularity_score']

0          1
1          1
2          1
3          1
4          1
          ..
1077593    1
1077594    1
1077595    1
1077596    1
1077597    1
Name: popularity_score, Length: 1077598, dtype: int32

In [32]:
df_with_popularity[df_with_popularity['popularity_score'] == 5]

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name,popularity_score
110,PB,2,spotify:track:6F609ICg9Spjrw1epsAnpa,Controlla,Drake,245226,Views,5
123,PB,15,spotify:track:0azC730Exh71aQlOt9Zj3y,This Is What You Came For,Calvin Harris,222160,This Is What You Came For,5
134,PB,26,spotify:track:6RsWqX8zABZLhZydXxEFOm,Can't Feel My Face,The Weeknd,213520,Beauty Behind The Madness,5
147,PB,39,spotify:track:5uDASfU19gDxSjW8cnCaBp,Chill Bill,Rob $tone,177184,Chill Bill,5
355,hype,3,spotify:track:7KOlJ92bu51cltsD9KU5I7,T-Shirt,Migos,242407,Culture,5
...,...,...,...,...,...,...,...,...
1076288,Sorry,14,spotify:track:1wHZx0LgzFHyeIZkUydNXq,Antidote,Travis Scott,262693,Rodeo,5
1076555,April 2017,65,spotify:track:3pXF1nA74528Edde4of9CC,Don't,Bryson Tiller,198293,T R A P S O U L,5
1076706,chill,81,spotify:track:14WWzenpaEgQZlqPq2nk4v,Work,Rihanna,219320,ANTI,5
1076744,n/a,29,spotify:track:03fT3OHB9KyMtGMt2zwqCT,Play It Again,Luke Bryan,226866,Crash My Party,5


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077598 entries, 0 to 1077597
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   playlist_name         1077598 non-null  object
 1   position_in_playlist  1077598 non-null  int64 
 2   track_id              1077598 non-null  object
 3   track_name            1077598 non-null  object
 4   artist_name           1077598 non-null  object
 5   duration_ms           1077598 non-null  int64 
 6   album_name            1077598 non-null  object
dtypes: int64(2), object(5)
memory usage: 57.6+ MB


In [6]:
df.head()

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP


In [10]:
track_name = df.drop_duplicates(subset=['track_name'])

In [11]:
track_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177749 entries, 0 to 1077595
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   playlist_name         177749 non-null  object
 1   position_in_playlist  177749 non-null  int64 
 2   track_id              177749 non-null  object
 3   track_name            177749 non-null  object
 4   artist_name           177749 non-null  object
 5   duration_ms           177749 non-null  int64 
 6   album_name            177749 non-null  object
dtypes: int64(2), object(5)
memory usage: 10.8+ MB


In [12]:
track_name.head(150)

,playlist_name,position_in_playlist,track_id,track_name,artist_name,duration_ms,album_name
0,Wedding Music,0,spotify:track:2rb4cO7RczQFSvpjTJ4C2P,Always Forever,Phil Wickham,281880,Phil Wickham
1,Wedding Music,1,spotify:track:017nSBNU2XHwMV0NCWZCqg,Divine Romance,Phil Wickham,298026,Phil Wickham
2,Wedding Music,2,spotify:track:0W5TB5VNs0J16suh3r67P1,Messiah / You're Beautiful,Phil Wickham,293720,Cannons
3,Wedding Music,3,spotify:track:1gBnG1MiTNBBVzmuwP7Wii,Love Is Not A Fight,Warren Barfield,222986,Worth Fighting For
4,Wedding Music,4,spotify:track:0b99xsUKkETGwZGzpX987r,When I Say I Do,Matthew West,246000,Hold You Up EP
...,...,...,...,...,...,...,...
147,PB,39,spotify:track:5uDASfU19gDxSjW8cnCaBp,Chill Bill,Rob $tone,177184,Chill Bill
148,PB,40,spotify:track:0L7loeTIg6akW84aEQZpCT,Light It Up (feat. Nyla & Fuse ODG) - Remix,Major Lazer,166138,Peace Is The Mission: Extended
149,PB,41,spotify:track:4nEfACF7NzZDxjEjLrw6Px,Party Like We're Animals - Radio Edit,Klaas,191453,Party Like We're Animals
150,PB,42,spotify:track:6HgWcisb1icZ70GNkFX9G1,This Is Dirty,DVBBS,295786,We Were Young


In [13]:
artists = df.drop_duplicates(subset=['artist_name'])

In [14]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45454 entries, 0 to 1077561
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         45454 non-null  object
 1   position_in_playlist  45454 non-null  int64 
 2   track_id              45454 non-null  object
 3   track_name            45454 non-null  object
 4   artist_name           45454 non-null  object
 5   duration_ms           45454 non-null  int64 
 6   album_name            45454 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.8+ MB


In [15]:
playlists = df.drop_duplicates(subset=['playlist_name'])

In [16]:
playlists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8650 entries, 0 to 1077551
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   playlist_name         8650 non-null   object
 1   position_in_playlist  8650 non-null   int64 
 2   track_id              8650 non-null   object
 3   track_name            8650 non-null   object
 4   artist_name           8650 non-null   object
 5   duration_ms           8650 non-null   int64 
 6   album_name            8650 non-null   object
dtypes: int64(2), object(5)
memory usage: 540.6+ KB
